In [167]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr

import rioxarray

In [168]:
data_dir = Path('../data/outputs')
sites_dir = data_dir / "sites"
sites_metrics_dir = sites_dir / "metrics" / "x1-y1-z1"
sites_netcdf_dir = sites_metrics_dir / "net_cdf"
sites_grid_tif_dir = sites_metrics_dir / "grid_tif"
sites_grid_tif_dir.mkdir(parents=True, exist_ok=True)
sites_voxel_tif_dir = sites_metrics_dir / "voxel_tif"
sites_voxel_tif_dir.mkdir(parents=True, exist_ok=True)


plots_dir = data_dir / "plots"
plot_metrics_dir = plots_dir / "metrics" / "x1-y1-z1"
plot_metrics_dir.mkdir(parents=True, exist_ok=True)
plot_net_cdf_dir = plot_metrics_dir / "net_cdf"
plot_net_cdf_dir.mkdir(parents=True, exist_ok=True)
plot_grid_tif_dir = plot_metrics_dir / "grid_tif"
plot_grid_tif_dir.mkdir(parents=True, exist_ok=True)
plot_voxel_tif_dir = plot_metrics_dir / "voxel_tif"
plot_voxel_tif_dir.mkdir(parents=True, exist_ok=True)

In [169]:
sites_gdf = gpd.read_file(sites_dir / "sites.geojson")
site_ids = sites_gdf['site'].to_list()

In [170]:
plots_gdf = gpd.read_file(plots_dir / "plots.geojson")
plot_site_geom_tuples = list(
    zip(
        plots_gdf["site_plot_id"].astype(str),
        plots_gdf["site"].astype(str),
        plots_gdf["geometry"],
    )
)

In [181]:
def read_site_metrics(site_id: str) -> xr.Dataset:
    metrics_path = sites_netcdf_dir / f"{site_id}.nc"
    ds = xr.open_dataset(metrics_path, decode_coords="all")
    return ds

def clip_metrics(ds: xr.Dataset, geom) -> xr.Dataset:
    clipped = ds.rio.clip([geom])
    return clipped

def save_grid_metrics(metrics: xr.Dataset, id: str, output_dir: Path):
    output_path = output_dir / f"{id}.tif"
    grid_metric_keys = [k for k in metrics.data_vars if metrics[k].ndim == 2]
    grid_metrics = metrics[grid_metric_keys]
    
    grid_metrics.rio.to_raster(output_path)

def flatten_voxel_metrics(metrics: xr.Dataset) -> list[xr.Dataset]:
    voxel_metric_keys = [k for k in metrics.data_vars if metrics[k].ndim == 3]
    voxel_metrics = metrics[voxel_metric_keys]
    
    flatted_datasets = []

    for metric in voxel_metrics.data_vars:
        # Select the variable as a new dataset
        ds_var = voxel_metrics[[metric]]
        
        # Create a new dataset with one variable per z slice
        new_vars = {}
        for i, z_val in enumerate(ds_var['z'].values):
            # Select the 2D slice at this z
            data_2d = ds_var[metric].sel(z=z_val)
            # Name for the new variable
            new_var_name = f"z{i}"
            # Add to dict
            new_vars[new_var_name] = (('y', 'x'), data_2d.values)
        
        # Build the new dataset
        ds_split = xr.Dataset(new_vars, coords={'y': ds_var['y'], 'x': ds_var['x']})
        ds_split.attrs = voxel_metrics.attrs
        ds_split.attrs['metric'] = metric
        
        flatted_datasets.append(ds_split)

    return flatted_datasets

def save_voxel_metrics(metrics: xr.Dataset, id: str, output_dir: Path):
    flatted_datasets = flatten_voxel_metrics(metrics)
    for ds in flatted_datasets:
        metric = ds.attrs['metric']
        output_path = output_dir / f"{id}_{metric}.tif"
        ds.rio.to_raster(output_path)

def read_and_clip_plot_metrics(plot_id: str, site_id: str, geom) -> xr.Dataset:
    ds = read_site_metrics(site_id)
    clipped = clip_metrics(ds, geom)
    clipped.attrs['site_plot_id'] = plot_id
    clipped = clipped.transpose('y', 'x', 'z')

    z_max = clipped['max_height'].max().item()
    clipped = clipped.sel(z=slice(0, np.ceil(z_max)))

    clipped.to_netcdf(plot_net_cdf_dir / f"{plot_id}.nc")
    save_grid_metrics(clipped, plot_id, plot_grid_tif_dir)
    save_voxel_metrics(clipped, plot_id, plot_voxel_tif_dir)
    return clipped

def read_and_save_site_metrics(site_id: str):
    ds = read_site_metrics(site_id)
    ds = ds.transpose('y', 'x', 'z')
    ds.load()
    ds.close()
    # ds.to_netcdf(sites_netcdf_dir / f"{site_id}.nc")
    save_grid_metrics(ds, site_id, sites_grid_tif_dir)
    save_voxel_metrics(ds, site_id, sites_voxel_tif_dir)
    return ds

In [177]:
plot = read_and_clip_plot_metrics(*plot_site_geom_tuples[1])
plot

<xarray.Dataset> Size: 6MB
Dimensions:              (x: 55, y: 57, z: 19)
Coordinates:
  * x                    (x) float64 440B 4.631e+05 4.631e+05 ... 4.631e+05
  * y                    (y) float64 456B 5.26e+06 5.26e+06 ... 5.26e+06
    spatial_ref          int64 8B 0
  * z                    (z) float64 152B 0.0 1.0 2.0 3.0 ... 16.0 17.0 18.0
Data variables: (12/40)
    num_points           (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    num_pulses           (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    total_weight         (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    ground_count         (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    ground_weight        (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    veg_count            (y, x) float64 25kB nan nan nan nan ... nan nan nan nan
    ...                   ...
    vox_enter_weight     (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
    vox_exit_weight      (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
    vox_lgap             (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
    vox_lgap_weight      (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
    vox_lcapture         (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
    vox_lcapture_weight  (y, x, z) float64 477kB nan nan nan nan ... nan nan nan
Attributes:
    xy_bin_size:   1
    z_bin_size:    1
    site_id:       AGG_O_01
    site_plot_id:  AGG_O_01_P2

In [182]:
read_and_save_site_metrics(site_ids[0])

<xarray.Dataset> Size: 413MB
Dimensions:              (x: 378, y: 175, z: 75)
Coordinates:
  * x                    (x) float64 3kB 4.63e+05 4.63e+05 ... 4.633e+05
  * y                    (y) float64 1kB 5.26e+06 5.26e+06 ... 5.26e+06 5.26e+06
  * z                    (z) float64 600B 0.0 1.0 2.0 3.0 ... 72.0 73.0 74.0
    spatial_ref          int64 8B 0
Data variables: (12/40)
    num_points           (y, x) float64 529kB nan nan nan nan ... nan nan nan
    num_pulses           (y, x) float64 529kB nan nan nan nan ... nan nan nan
    total_weight         (y, x) float64 529kB nan nan nan nan ... nan nan nan
    ground_count         (y, x) float64 529kB nan nan nan nan ... nan nan nan
    ground_weight        (y, x) float64 529kB nan nan nan nan ... nan nan nan
    veg_count            (y, x) float64 529kB nan nan nan nan ... nan nan nan
    ...                   ...
    vox_enter_weight     (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
    vox_exit_weight      (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
    vox_lgap             (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
    vox_lgap_weight      (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
    vox_lcapture         (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
    vox_lcapture_weight  (y, x, z) float64 40MB nan nan nan nan ... nan nan nan
Attributes:
    xy_bin_size:  1
    z_bin_size:   1
    site_id:      AGG_O_01

In [183]:
for site in site_ids:
    print(f"Processing site {site}")
    read_and_save_site_metrics(site)

Processing site AGG_O_01
Processing site AGG_O_05
Processing site AGG_O_07
Processing site AGG_Y_02
Processing site AGG_Y_03
Processing site AGG_Y_04
Processing site EPO_O_04
Processing site EPO_O_13
Processing site EPO_Y_10
Processing site EPO_Y_15
Processing site EPO_Y_36
Processing site EPO_Y_37
Processing site EPY_O_01
Processing site EPY_O_07
Processing site EPY_O_09
Processing site EPY_Y_03
Processing site EPY_Y_04
Processing site EPY_Y_05
Processing site NRM_31
Processing site NRM_O_15
Processing site NRM_O_19
Processing site NRM_Y_39
Processing site NRM_Y_41
Processing site NRM_Y_69
Processing site NRO_138
Processing site NRO_174
Processing site NRO_190
Processing site NRO_52
Processing site NRO_O_56
Processing site NRO_Y_82
Processing site NRY_O_14
Processing site NRY_O_20
Processing site NRY_O_25
Processing site NRY_Y_03
Processing site NRY_Y_12
Processing site NRY_Y_15
Processing site PPO_O_05
Processing site PPO_O_09
Processing site PPO_Y_07
Processing site PPO_Y_09
Process

In [184]:
for p in plot_site_geom_tuples:
    print(f"Processing plot {p[0]} from site {p[1]}")
    read_and_clip_plot_metrics(*p)

Processing plot AGG_O_01_P1 from site AGG_O_01
Processing plot AGG_O_01_P2 from site AGG_O_01
Processing plot AGG_O_01_P3 from site AGG_O_01
Processing plot AGG_O_01_P4 from site AGG_O_01
Processing plot AGG_O_01_P5 from site AGG_O_01
Processing plot AGG_O_05_P1 from site AGG_O_05
Processing plot AGG_O_05_P2 from site AGG_O_05
Processing plot AGG_O_05_P3 from site AGG_O_05
Processing plot AGG_O_05_P4 from site AGG_O_05
Processing plot AGG_O_05_P5 from site AGG_O_05
Processing plot AGG_O_07_P1 from site AGG_O_07
Processing plot AGG_O_07_P2 from site AGG_O_07
Processing plot AGG_O_07_P3 from site AGG_O_07
Processing plot AGG_O_07_P4 from site AGG_O_07
Processing plot AGG_O_07_P5 from site AGG_O_07
Processing plot AGG_Y_02_P1 from site AGG_Y_02
Processing plot AGG_Y_02_P2 from site AGG_Y_02
Processing plot AGG_Y_02_P3 from site AGG_Y_02
Processing plot AGG_Y_02_P5 from site AGG_Y_02
Processing plot AGG_Y_02_P4 from site AGG_Y_02
Processing plot AGG_Y_03_P1 from site AGG_Y_03
Processing pl